# Further topics on Python 3

## Contents

* Classes
  * Static attributes
  * Properties
  * Inheritance
* Scopes - The lifetime of variables
* Testing  
* Documenting your code
* Reading and writing files
* Using and writing modules  

## Before we start, some updates

Assignment 3:
* Deadline for assignment 3 is today.

Assignment 4:
* Published today!
* 3 weeks deadline followed by a 1 week peer-review
* Topic: Numerical Python (NumPy) and integration with C

## Lecture overview for the next weeks

* 15 Sept    -  Classes / Unit testing
* 22 Sept    -  Numpy / Matplotlib / Profiling
* 29 Sept    -  Mixed programming / Cython
* 06 October -  Ipython Notebook (Guest lecture by Benjamin Ragan-Kelley)

# Classes in Python

## Classes in Python

In [21]:
class Car(object):
    """ Class docstring """
    
    def __init__(self, color):       # Constructor
        self.color = color           # Class variable
        self.sound = "Roaar"

    def start(self):                 # member function
        print(self.sound)

* *Class attributes* can be variables and functions.

In [22]:
car = Car("blue")
car.start()

Roaar


## So what again is this `self` variable?

`self` is the instance object that Python automatically passes to the class instance's method as its first argument when called. 

In other words:

```python
car.start()
```
and
```
Car.start(self=car)
``` 
do the same!

Note: the name `self` is arbitrary - but a good choice in most cases.


### And why do we need it?

`self` is used to access other attributes or methods of the object from inside the method.
```python
class Car(object):
    # ...
    def start(self):          
        print(self.sound)  # accesses the variable of that car instance
```        

## Subclasses

* Python has a similar class concept as in Java and C++, but:
  * All functions are virtual
  * No private/protected variables (the effect can be "simulated")
  * This makes class programming easier and faster than in C++ and Java:
* Python supports single and multiple inheritance

## Implementing a subclass

Subclasses can be used to **inherit** all member functions/attribites from its Base class.

It is usefull to create **specialisation** of a class without code duplication.

For example, `Golf` is a special type of car:

In [33]:
class Golf(Car):
    def __init__(self, color):
        Car.__init__(self, color)         # Calls the Car constructor
                                          # self.color and self.sound now exist
        self.sound = "(Golf) RRROOAR"     # Change the sound

If a subclass function is not implemented, Python will try to call the function of the class:

In [35]:
golf = Golf("white")
golf.start()          # Calls the Car's start function.

(Golf) RRROOAR


We can add special Subclasses can also have additional functions:

In [61]:
class DieselGolf(Golf):
    def cheat(self):
        self.sound = "(Golf) SSsssss"

In [62]:
diesel_golf = DieselGolf("black")
diesel_golf.cheat()
diesel_golf.start()

(Golf) SSsssss


### The `object` class is the mother of all classes

All classes are subclasses of the `object` class:
```python
class Car(object):
    # ...
```

Let's see what functions the object class implements:

In [42]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

## Comment on object-orientation
Consider:

In [68]:
# this function works with any object that has a start function
def start(v):
    v.start()

start(car)

Roaar


* In C++/Java we would declare `v` as a `Car` reference and rely on `golf.start()` to call the virtual function `start` in `Golf`.
* The same works in Python, but we do not need inheritance and virtual functions here: `v.start()` will work for **any** object `v` that has a callable attribute `start` that takes no arguments.

## Testing on the class type

It can be usefull to check the type of an object.

For example if the object is an instance of a class:
```python
if isinstance(obj, Car): # True
    # Treat obj as a car
```

Test if a class is a subclass of another:
```python
if issubclass(Golf, Car):
    # ...
```

Test if two objects are of the same class:
```python
if car.__class__ is golf.__class__:
```
(`car.__class__` refers the class object of instance `car`)

## Private/non-public data

There is no technical way of preventing users from manipulating data and methods in an object:

In [74]:
car.start = lambda: print("---")
car.start()

---


However, there are some conventions: 
* Names **starting with one underscore** are treated as non-public ("protected").
* Names **starting with one double underscore** are considered strictly private (Python mangles class name with method name in this case: `obj.__some` has actually the name `_classname__some`).
* Names **starting and ending with double underscores** are special methods and attributes (discussed later)

Here is an example...

In [82]:
class MyClass(object):
    def __init__(self):
        self.a   = 2       # public var
        self._b  = 1       # non-public var
        self.__c = 3       # private var
        
    def __hidden(self):    # private function
        pass

and how to use it:

In [84]:
m = MyClass()
m.a
m._b
m._MyClass__c         # m.__c has been mangled
m._MyClass__hidden()  # m.__hidden has been mangled

You can list all public and private class asttributes with `dir`:

In [77]:
dir(m)

['_MyClass__c',
 '_MyClass__hidden',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_a',
 'b']

## Special attributes

Listing all methods and attributes with `dir` reveils that these instances have already some *special* attributes:

In [85]:
dir(car)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'color',
 'sound',
 'start']

Let's inspect some of these:

In [94]:
golf.__dict__

{'color': 'white', 'sound': '(Golf) RRROOAR'}

In [95]:
golf.__dict__

{'color': 'white', 'sound': '(Golf) RRROOAR'}

**Conclusion**: `__dict__` returns a dictionary of user-defined attributes.

Name of class, name of method:

In [96]:
golf.__class__          # class of object

__main__.Golf

In [97]:
golf.__class__.__name__ # name of class

'Golf'

## Special methods
* Special methods have leading and trailing double underscores.
* If implemented, Python calls these special  methods on certain operations. 

* Here are some operations defined by special methods:

In [58]:
str(a)            # calls a.__str__()
                  # also called with print(a)!
len(a)            # calls a.__len__()
c = a*b           # calls c = a.__mul__(b)
a = a+b           # calls a = a.__add__(b)
a += c            # calls a.__iadd__(c)
d = a[3]          # calls d = a.__getitem__(3)
a[3] = 0          # calls a.__setitem__(3, 0)
f = a(1.2, True)  # calls f = a.__call__(1.2, True)
if a:             # calls if a.__len__()> 0: or if a.__nonzero__():
a == b            # calls a.__eq__(self, b)

IndentationError: expected an indented block (<ipython-input-58-0dfdac666397>, line 11)

## Example: functions with extra parameters

Suppose we need a function of `x` and `y` with three additional parameters `a`, `b`, and `c`:

In [21]:
def f(x, y, a, b, c):
    return a + b*x + c*y*y

Suppose we need to send this function to another function

In [22]:
def gridvalues(f, xcoor, ycoor, file):
    for i in range(len(xcoor)):
        for j in range(len(ycoor)):
            fval = f(xcoor[i], ycoor[j])
            file.write('%g %g %g\n' % (xcoor[i], ycoor[j], fval))

* `func` is expected to be a function of `x` and `y` only (many libraries make such assumptions!)
* How can we send our `f` function to `gridvalues`?

## Possible (inferior) solution
**Bad solution 1**: global parameters

```python
global a, b, c
#...
def f(x, y):
    return a + b*x + c*y*y

#...
a = 0.5;  b = 1;  c = 0.01
gridvalues(f, xcoor, ycoor, somefile)
```

**Problem**: Global variables are usually considered evil!

## Possible (inferior) solution
**Bad solution 2**: keyword arguments with default values

```python
def f(x, y, a=0.5, b=1, c=0.01):
    return a + b*x + c*y*y

# ...
gridvalues(f, xcoor, ycoor, somefile)
```

**Problem**: Useless for other values of `a`, `b`, `c`

## Solution: class with `__call__` operator

Make a class with function behavior instead of a pure function.
  2. Make the parameters class attributes.
  3. Implement the special `__call__`  function.

In [23]:
class F(object):
    def __init__(self, a=1, b=1, c=1):
        self.a = a
        self.b = b
        self.c = c

    def __call__(self, x, y):    # special method!
        return self.a + self.b*x + self.c*y*y

Now, instances can be called as ordinary functions, but with *x* and *y* as the only formal arguments:

In [24]:
f = F(a=0.5, c=0.01)
# can now call f as
v = f(0.1, 2)
# ...
gridvalues(f, xcoor=[], ycoor=[], file="somefile")

## Class variables
Static data (or class variables) are common to all class instances.

In [2]:
class Point(object):
    " A class representing a 2D point "

    counter = 0 # class variable, counts number of instances
    
    def __init__(self, x, y):
        self.x = x
        self.y = y;
        Point.counter += 1

In [5]:
for i in range(1000):
        p = Point(i*0.01, i*0.001)

Class variables can be accessed without an instance:

In [6]:
Point.counter     

2000

... or with with an instance:

In [7]:
p.counter      

2000

## Static methods
Python also allow static class methods (that is, methods that can be called without having an instance):

In [118]:
class Point(object):
    " A class representing a 2D point "
    _counter = 0
    
    def __init__(self, x, y):
        self.x = x; self.y = y
        Point._counter += 1
        
    def _ncopies():  # No need for a self argument for static methods
        return Point._counter

    ncopies = staticmethod(_ncopies)

`ncopies` can be called directly from the class:

In [119]:
Point.ncopies()

0

... or from an instance

In [120]:
p = Point(0, 0)
p.ncopies()

1

In [121]:
Point.ncopies()

1

# Properties

## Properties example

How can we prevent invalid x, y values?

In [122]:
p = Point(x=0, y=0)

p.x = "should not be allowed" 
print(p.x)  

should not be allowed


### **Idea 1**: Implement get/set functions

In [140]:
class SafePoint(Point):
    def set_x(self, x):
        if isinstance(x, float):
            self._x = x         # store x value is a non-public variable
        else:
            print("Invalid value for x coordinate")
 
    def get_x(self):
        return self._x

In [141]:
point = SafePoint(1, 1)
point.set_x(True)   

Invalid value for x coordinate


In [143]:
point.set_x(3.14) 
print(point.get_x())

3.14


**Advantage**: get/set functions allow finer access control (e.g. type-checking, read-only variables). 

**Disadantage**: Tedious to write and not very Pythonic!

## The solution: Properties
Python has "intelligent" assignment operators, known as
*properties*. 

With properties, assignments may imply a function call:

```python
point.set_x(data)
data = point.get_x()
```

can be made equivalent to

```python
point.x = data
data = point.x
```

Creating properties is simple:

In [153]:
class SafePoint(Point):
    def set_x(self, x):
        if isinstance(x, float):
            self._x = x  
        else:
            print("Invalid value for x coordinate")
 
    def get_x(self):
        print("Retrieving x value")
        return self._x

    x = property(fget=get_x, fset=set_x)

Example:

In [152]:
point = SafePoint(1.0, 1.0)
point.x = 2.0
#x = point.x

## Attribute access - recommended style

* Use **direct access** if user is allowed to read *and* assign values to the attribute.
* Use **properties** to restrict access, with a corresponding underlying non-public class attribute.
* Use **properties** when assignment or reading requires a set of associated operations.
* **Never use get/set** functions explicitly.

## Scope
The scope defines how long variables in Python *live*.

#### Local scope
Function arguments and variables declared inside the function have *local scope*. Once the function finishes, these variables are freed.

#### Global scope
Variables defined outside the function have *global scope*. These variables can be accessed and changed by the function and are accessible after the function returns.

## Code example for scopes

In [8]:
def f():
    a = 1  # local scope
    print(a)
 
f()
a

1


NameError: name 'a' is not defined

## Code example for scopes

In [9]:
# global scope
a = 1 

def f():
    print(a)

f()
a = 2
f()

1
2


## Code example for scopes

In [13]:
# global scope
a = 1 

def f():
    print(a)
    a_ = 3
    print(a_)

a = "s"
f()

s
3


## Code example for scopes

In [20]:
# global scope
a = ["Hello"]

def f():
    def g():
        a.append("world?")
    g()

f()
print(a)
b

['Hello', 'world?']


NameError: name 'b' is not defined

## Code example for scopes

In [41]:
# global scope
a = 1 

def f(x):
    a = 2             # local variable

class B(object):
    b = 3             # static class attribute

    def __init__(self):
        self.a = 3    # class attribute

    def scopes(self):
        a = 4         # local (method) variable

## Namespaces for exec and eval
`exec` and `eval` may take dictionaries for the global and local namespace:

In [42]:
code = "a=1"
exec(code, globals(), locals())
expr = "a+1"
eval(expr, globals(), locals())

2

Example:

In [43]:
a = 8;  b = 9
d = {'a':1, 'b':2}
eval('a + b', d)  # yields 3

3

and

from math import *
d['b'] = pi
eval('a+sin(b)', globals(), d)  # yields 1

Creating such dictionaries can be handy

# Testing in Python

## Why should we test?

* To check correctness of software.
* To ensure that future changes do not break functionality.
* To check if the software runs succesfully in a different environment (newer Python version, upgraded libraries, different operating system)

## A few options in Python

* [Unittest](https://docs.python.org/3/library/unittest.html)

* [Doctest](https://docs.python.org/3/library/doctest.html)
* [Py.test](http://pytest.org/) (will be used here)

## How to use py.test

Say you have a function `func` in a file that needs testing:

In [15]:
# script.py 
def func(x):
    if x < 0:
        return -x
    else:
        return x

Create a associated test file `test_script.py`:

In [16]:
# test_script.py
from script import func  # Import the function 

def test_funcs():        # py.test will automatically run all functions starting with test_
    assert func(-3) == 3 # Add some tests here...
    assert func(5) == 5  # If one of the assert's are false,
    assert func(0) == 0  # the test will fail

In [17]:
!py.test test_script.py -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.12, pytest-2.8.1, py-1.4.30, pluggy-0.3.1 -- /usr/bin/python
cachedir: .cache
rootdir: /home/sf1409/Documents/inf3331/resources-16/lectures/04-python-summary2, inifile: 
plugins: xdist-1.13.1
collected 1 items 

test_script.py::test_func PASSED

=========================== 1 passed in 0.01 seconds ===========================


# Using py.test
Now, lets make a mistake in our func implementation ...

In [18]:
!sed -i "s/-x/x/g" script.py
!cat script.py

def func(x):
    if x < 0:
        return x
    else:
        return x


and run the tests again

In [19]:
!py.test test_script.py -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.12, pytest-2.8.1, py-1.4.30, pluggy-0.3.1 -- /usr/bin/python
cachedir: .cache
rootdir: /home/sf1409/Documents/inf3331/resources-16/lectures/04-python-summary2, inifile: 
plugins: xdist-1.13.1
collected 1 items 

test_script.py::test_func FAILED

=================================== FAILURES ===================================
__________________________________ test_func ___________________________________

    def test_func():
>       assert func(-3) == 3
E       assert -3 == 3
E        +  where -3 = func(-3)

test_script.py:4: AssertionError
=========================== 1 failed in 0.01 seconds ===========================


## Good practices
* Add new test while you develop new features.
* Make each test an unique stand alone example.
* Making tests resource undemanding.
* Run test suite before each commit-push.
* Make test function names descriptive.
* Quick way to learn other peoples code is through test suits.

In [ ]:
# Modules and packages
## Use modules to organize your program logically
### What is a Python module?
A module is a file consisting of Python code. A module can define functions, classes and variables. A module can also include runnable code.

In [ ]:
### What is it good for?
  * Split the code into several files for easier maintenance.
  * Group related code into a module.
  * Share common code between scripts.
  * Publish modules on the web for other people to use.

In [ ]:
## Using modules

Import the module called `sys` and access its `argv` variable:

```python
import sys
x = float(sys.argv[1])
```

In [ ]:
Import module member `argv` into current namespace:

```python
from sys import argv
x = float(argv[1])
```

In [ ]:
Import everything from `sys` (not recommended)

```python
from sys import

In [ ]:
Import everything from `sys` (not recommended)

```python
from sys import *
x = float(argv[1])

flags = ''
# Ooops, flags was also imported from sys, this new flags
# name overwrites sys.flags!
```

In [ ]:
Import `argv` under an alias:

```python
from sys import argv as a
x = float(a[1])
```

In [ ]:
## Making your own Python modules


 * Reuse scripts by wrapping them in classes or functions

 * Collect classes and functions in library modules

 * How? just put classes and functions in a file `MyMod.py`

 * Put `MyMod.py` in one of the directories where Python can find it
   (see next slide)

Examples:

```python
import MyMod
# or
import MyMod as M   # M is a short form
# or
from MyMod import *
# or
from MyMod import myspecialfunction, myotherspecialfunction
```

In [ ]:
## Document your module using docstrings

In [ ]:
"""
This module exemplifyies multi-line
doc strings.
"""
import sys
import collections

def somefunc():
    """ Function documentation """
    ...
    

In [ ]:
## How does Python find your modules?


Python has some "official" module directories, typically

* `/usr/lib/python3.5`
* `/usr/lib/python3.5/site-packages`
* current working directory

The environment variable `PYTHONPATH` may contain additional
directories with modules

```bash
> echo $PYTHONPATH
/home/simon/src/cbc.block:/home/simon/src/shape-optimisation:/home/simon/src/uptide:/home/simon/src/OpenTidalFarm:
```

Python's `sys.path` list contains the directories where Python
searches for modules, and
`sys.path` contains "official" directories, plus those in `PYTHONPATH`

In [ ]:
## Search path for modules can be set in the script

Add module path(s) directly to the `sys.path` list:
    

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.environ['HOME'], 'python', 'lib'))

# ...
import MyMod

In [ ]:
## Packages (1)

 * A set of modules can be collected in a *package*

 * A package is organized as module files in a directory tree

 * Each subdirectory has a file `__init__.py` (can be empty)

 * Documentation: [Section 6 in the Python Tutorial](https://docs.python.org/3/tutorial/modules.html)
    

In [ ]:
## Packages (2)

Example directory tree:
    

In [ ]:
```bash
MyMod
   __init__.py
   numerics
       __init__.py
       pde
           __init__.py
           grids.py     # contains fdm_grids object
```

In [ ]:
Can import modules in the tree like this:
    

In [ ]:
```python
from MyMod.numerics.pde.grids import fdm_grids

grid = fdm_grids()
grid.domain(xmin=0, xmax=1, ymin=0, ymax=1)
...
```

In [ ]:
## Test block in a module


Module files can have a test/demo section at the end:
    

In [ ]:
if __name__ == '__main__':
    infile = sys.argv[1]; outfile = sys.argv[2]
    for i in sys.argv[3:]:
        create(infile, outfile, i)
        

In [ ]:
* The block is executed *only if* the module file is run as a program (not if imported by another script)
* The tests at the end of a module often serve as good examples on the usage of the module

In [ ]:
## Public/non-public module variables

Python convention: add a leading underscore to non-public functions and (module) variables
    

In [ ]:
_counter = 0

def _filename():
    """Generate a random filename."""
    ...
    